## Downloading fasttext

In [1]:
from scipy import spatial
import numpy as np
from scipy.spatial.distance import cosine
import numpy as np
# !git clone https://github.com/facebookresearch/fastText.git
# !cd fastText
# !pip install fastText
import fasttext
import fasttext.util
import pandas as pd
import ast
from numpy import mean
import pickle
# download an english model of fasttext
fasttext.util.download_model('en', if_exists='ignore')  # English
model = fasttext.load_model('cc.en.300.bin')

## Reading files

In [2]:
reviews=pd.read_csv("FPS_reviews_lemmatized.csv")
with open("control_recommendation_ids_all.pkl", "rb") as fp1:
  control_id = pickle.load(fp1)

In [3]:
import pickle
from tqdm import tqdm
import ast
with open("recommendation_id.pkl", "rb") as fp2:
  recommendation_ids = pickle.load(fp2)
recommendation_ids = [str(elem) for elem in tqdm(recommendation_ids)]
control_id=[[str(elem) for elem in inner_list] for (inner_list) in tqdm(control_id)]

100%|█████████████████████████████████| 920536/920536 [08:03<00:00, 1905.15it/s]


## Computing fastText and the cosine-similarity matrices

Saving in chunks due to the massive size of the control files

In [5]:
model = fasttext.load_model('cc.en.300.bin')

# getting the fastText word vectors for each reviews
def get_vector(s):
  return (model.get_sentence_vector(str(s)))

# getting the fastText word vectors for across all reviews and saving it in the form of a dictionary
def get_vector_dict(reviews):
  review_ids=list(reviews["recommendation_id"])
  review=list(reviews["review"])
  vector={}
  for i in tqdm(range(len(review))):
    review_id=str(review_ids[i])
    fasttext_vector=get_vector(review[i])
    vector[review_id]=fasttext_vector
  return(vector)

# calculating the cosine-similarity score 
def cosine_similarity(embedding_1, embedding_2):
  # Calculate the cosine similarity of the two embeddings.
  sim = 1 - cosine(embedding_1, embedding_2)
  return(sim)

# getting the cosine similarity matrix for each review
def similiarity_matrix(list_of_reviews,recommendation_id,comments_vector):
  review_id=str(recommendation_id)
  review_vector=list(comments_vector[review_id])
  matrix=[]
  for i in range(0,len(list_of_reviews)):
    order_id=str(list_of_reviews[i])
    order_x=comments_vector[order_id]
    similarity=cosine_similarity(review_vector,order_x)
    matrix.append(similarity)
  return(matrix)

vector_dict=get_vector_dict(reviews)
# compiling all the similarity matrices 

def list_of_matrix(review_order_list,comments_vector):
  matrix_list=[]
  for i in (range(len(review_order_list))):
    recommendation_id=recommendation_ids[i]
    list_of_visible_matrices=similiarity_matrix(list(review_order_list[i]),recommendation_id,comments_vector)
    matrix_list.append(list_of_visible_matrices)
  return(matrix_list)

# Saving in chunks due to the massive size of the control files
current,total=0,0
while current<=len(recommendation_id):
    #print("At", current, "to", (current+1000))
    control_id_current=control_id[current:current+1000]
    i=str(current)+"_to_"+str(current+1000)
    control_matrix=list_of_matrix(control_id_current,vector_dict)
    # Open a new file for writing in binary mode
    file_name="control_matrix_{}.pickle".format(i)
    with open(file_name, 'wb') as f:
        # Use the pickle.dump() method to write the data to the file
        pickle.dump(control_matrix, f)
    current=current+1000
    total=total+1000

100%|██████████████████████████████| 1056940/1056940 [00:44<00:00, 23765.06it/s]
